### Convert All Rasters in Folder to Shapefile ###

In [ ]:
# Convert using GRASS GIS

import os
from gasp.to.shp  import rst_to_polyg
from gasp.oss     import lst_ff
from gasp.cpu.grs import run_grass
from gasp.oss import get_filename

GRASS_WORKSPACE = r'D:\indev\xyz'
GRASS_LOCATION  = 'conv_viena'
RST_FOLDER = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena'
OUT_FOLDER = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena'
SRS = 3857
RST_TEMPLATE = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena\v14_viena_2x2.tif'

# List Raster Files
rstFiles = lst_ff(RST_FOLDER, file_format='tif')

# Start GRASS GIS Session
grsbase = run_grass(
    GRASS_WORKSPACE, location=GRASS_LOCATION, srs=SRS)

import grass.script.setup as gsetup
gsetup.init(grsbase, GRASS_WORKSPACE, GRASS_LOCATION, 'PERMANENT')

from gasp.to.rst.grs import rst_to_grs
from gasp.cpu.grs.conf import rst_to_region
from gasp.to.shp.grs import grs_to_shp

# Set region
tmpRst = rst_to_grs(RST_TEMPLATE, get_filename(RST_TEMPLATE), as_cmd=True)
rst_to_region(tmpRst)

for rst in rstFiles:
    # Add data to GRASS GIS
    grsRst = rst_to_grs(rst, get_filename(rst), as_cmd=True)
    
    # Convert to Polygon
    shpGrs = rst_to_polyg(
        grsRst, grsRst + "_shp",
        rstColumn="value", gisApi='grasscmd'
    )
    
    # Export to File
    grs_to_shp(shpGrs, os.path.join(OUT_FOLDER, grsRst + '.shp'), 'area')

### All Nc in Folders to tiff ###

In [ ]:
from gasp.to.rst.gdl import folder_nc_to_tif

folder_nc_to_tif(
    r'D:\gis\UNDERSee\sentinel_3', r'D:\gis\UNDERSee\sentinel_3'
)

### DB Table to XLS File ###

In [ ]:
from gasp.to import db_to_tbl

QS = (
    "SELECT link as url FROM facedata WHERE "
    "page_ref='DiariodeNoticias.pt' AND "
    "link LIKE '%%www.dn.pt%%'"
)

XLS = '/home/jasp/expvgi/dn_noticias.xlsx'
SHEETS = 'news'

CON_DB = {
    'HOST' : 'localhost', 'USER': 'postgres',
    'PASSWORD' : 'admin', 'PORT' : '5432',
    'DATABASE' : 'dsn_collect'
}

DB_API = 'psql'

db_to_tbl(CON_DB, QS, XLS, sheetsNames=SHEETS, dbAPI=DB_API)

In [ ]:
from gasp3.to.sql import tbl_to_db
from gasp3.pyt.oss import get_filename

conParam   = {
    "HOST" : "localhost", "USER" : "postgres", "PORT" : "5432",
    "PASSWORD" : "admin", "DATABASE" : "dsn_collect"
}

TBLS = [
    '/home/jasp/expvgi/GeoTmLnh_data_collector/geoname/cities_1.dbf',
    '/home/jasp/expvgi/GeoTmLnh_data_collector/geoname/concelhos.dbf',
    '/home/jasp/expvgi/GeoTmLnh_data_collector/geoname/freguesias.dbf',
    '/home/jasp/expvgi/GeoTmLnh_data_collector/geoname/lugares_pt_1.dbf'
]

for t in TBLS:
    tbl_to_db(t, conParam, get_filename(t), api_db='psql')